In [10]:
!pip install tkPDFViewer
!pip install pdf2image
!pip install PyAutoGUI
!pip install --upgrade pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from tkinter import *
from tkinter import filedialog
from tkPDFViewer import tkPDFViewer as pdf 
import os
import threading
from pdf2image import convert_from_path 
from PIL import Image
import cv2
import numpy as np
import pyautogui
import threading

def blink():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')


    # Variable store execution state
    first_read = True

    # Starting the video capture
    cap = cv2.VideoCapture(1)
    ret, img = cap.read()
    blink=[]
    num=0
    take=False
    while (ret):
        ret, img = cap.read()
        # Converting the recorded image to grayscale
       # img=cv2.resize(img,(1000,1000),fx=0,fy=0,interpolation=cv2.INTER_CUBIC)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Applying filter to remove impurities
        gray = cv2.bilateralFilter(gray, 5, 1, 1)

        # Detecting the face for region of image to be fed to eye classifier
        faces = face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(150, 150))
        if (len(faces) > 0):
            for (x, y, w, h) in faces:
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

                # roi_face is face which is input to eye classifier
                roi_face = gray[y:y + h, x:x + w]
                roi_face_clr = img[y:y + h, x:x + w]
                eyes = eye_cascade.detectMultiScale(roi_face, 1.3, 5, minSize=(23, 23))
                print(len(eyes))
                blink.append(len(eyes))
                # Examining the length of eyes object for eyes
                if (len(eyes) == 2):
                    # Check if program is running for detection
                    if (first_read):
                        if len(blink)>30:
                            count=0
                            for b in range(2,28):
                                if  blink[-b]!=2:
                                    count+=1
                            if count<10:
                                take=True
                        cv2.putText(img,
                                    "Eye detected press s to begin",
                                    (70, 70),
                                    cv2.FONT_HERSHEY_PLAIN, 3,
                                    (0, 255, 0), 2)
                    else:
                        cv2.putText(img,
                                    "Eyes open!", (70, 70),

                                    cv2.FONT_HERSHEY_PLAIN, 2,
                                    (255, 255, 255), 2)
                elif len(eyes) == 1:
                    if len(blink)>30:
                        ok=True
                        count=0
                        for b in range(2,28):
                            if  blink[-b]!=1:
                                count+=1
                        if count>10:
                            ok=False
                        if first_read and ok:
                            print('one eye')
                            cv2.putText(img,
                                        "one Eye detected press s to begin",
                                        (70, 70),
                                        cv2.FONT_HERSHEY_PLAIN, 3,
                                        (0, 255, 0), 2)
                            if take:
                                take=False
                                takeScreenshot(num)
                                num+=1

                else:
                    if (first_read):
                        # To ensure if the eyes are present before starting
                        cv2.putText(img,
                                    "No eyes detected", (70, 70),
                                    cv2.FONT_HERSHEY_PLAIN, 3,
                                    (0, 0, 255), 2)
                    else:
                        # This will print on console and restart the algorithm
                        print("Blink detected--------------")
                        cv2.waitKey(3000)
                        first_read = True

        else:
            cv2.putText(img,
                        "No face detected", (100, 100),
                        cv2.FONT_HERSHEY_PLAIN, 3,
                        (0, 255, 0), 2)

        # Controlling the algorithm with keys
        cv2.imshow('img', img)
        a = cv2.waitKey(1)
        if (a == ord('q')):
            break
        elif (a == ord('s') and first_read):
            # This will start the detection
            first_read = False

    cap.release()
    cv2.destroyAllWindows()

back=[255,255,255]
font=[0,0,0]
def pdf2img1(filename):
#     global filename
#     filename = filedialog.askopenfilenames()
#     filename = str(filename)
#     filename = filename[1:len(filename)-2]
    global images
    images = convert_from_path(filename)
    for i in range(len(images)):
        images[i].save('page'+str(i)+'.jpg','JPEG')
        
def convertColor():
    image_list=[]
    for i in range(len(images)):
        img=cv2.imread('page'+str(i)+'.jpg')
        GRY = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img=cv2.merge([GRY,GRY,GRY])
        ret,thresh1 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        img=thresh1
        print(back)
        img[np.where(( img==[0,0,0]).all(axis=2))]=font
        img[np.where(( img==[255,255,255]).all(axis=2))]=back
        cv2.imwrite('copy_page'+str(i)+'.jpg' , img)
    pdf2imag()
    
def pdf2imag():
    image=[]
    for i in range(len(images)):
        img=Image.open(r'copy_page'+str(i)+'.jpg')
        img=img.convert('RGB')
        image.append(img)
    img=image[0]
    img.save(r'target.pdf',save_all=True,append_images=image)

    
def eye_confort(vul,bol=1):
    if bol :
        for i in range(len(images)):
            img= cv2.imread('copy_page'+str(i)+'.jpg')
            b,g,r=cv2.split(img)
            b=np.dot(b,vul)
            b=b.astype(int)
            img[:,:,0]=b
            cv2.imwrite('copy_page'+str(i)+'.jpg' , img)
    pdf2imag()
    
def takeScreenshot(num):
    im1 = pyautogui.screenshot()
    # you take path to save in function save()
    im1.save(r'C:\Users\123\Desktop\eye tracking\screenshot'+str(num)+'.png')


    
def Open():
    filename = (filedialog.askopenfilename(initialdir=os.getcwd(),title='selsct pdf file',
                                           filetype=(('PDF File','.pdf'),('PDF File','.PDF'),('All file','.txt'))))
    pdf2img1(filename)
     
    pdffile = pdf.ShowPdf()
    file_pdf = pdffile.pdf_view(root,pdf_location=filename)
    file_pdf.pack()
    file_pdf.place(relx=0.0,rely=0.15)


def Back():
    print([blue.get(),green.get(),red.get()])
    
    back[0]=blue.get()
    back[1]=green.get()
    back[2]=red.get()
    print (back)
    convertColor()


def Font():
    print([blue.get(),green.get(),red.get()])
    font[0]=blue.get()
    font[1]=green.get()
    font[2]=red.get()
    print(back)
    convertColor()


def confort():
    print(eyeconfort.get())
    z = (100 - eyeconfort.get())/100
    print(z)
    eye_confort(z)


root =Tk()
root.geometry('630x700')
root.title('pdf viewer')

#GetPath = Button(root,text ="ScreenShot",command = blink,width=7,bd=5)

Open = Button(root,text ="Open",command = Open,width=7,bd=5)

Close = Button(root,text='EyeConf',command = confort ,width=7,bd=5)

Background = Button(root,text='BackGro',command=Back,width=7,bd=5)

Font = Button(root,text='Font',command=Font,width=7,bd=5)


eyeconfort = Scale(root, from_=0, to=100, orient=HORIZONTAL,length=285,background='white')
eyeconfort.pack()

red = Scale(root, from_=0, to=255, orient=HORIZONTAL,length=195,background='red')
red.pack()

green = Scale(root, from_=0, to=255, orient=HORIZONTAL,length=195,background='green')
green.pack()

blue = Scale(root, from_=0, to=255, orient=HORIZONTAL,length=195,background='blue')
blue.pack()

#treadingg
t=threading.Thread(target=blink)
t.setDaemon(True)
t.start()
#t.stare()

#GetPath.place(relx = 0.0,rely = 0.0)

Open.place(relx = 0.1, rely = 0.0)

Close.place(relx = 0.2, rely = 0.0)

Background.place(relx = 0.3, rely = 0.0)

Font.place(relx = 0.4, rely = 0.0)

eyeconfort.place(relx=0.52, rely=0.0)

red.place(relx=0.0, rely=0.07)

green.place(relx=0.33, rely=0.07)

blue.place(relx=0.66, rely=0.07)


mainloop()
#blink()


root.mainloop()

AttributeError: ignored